# Data extraction methodology

## Indices
* preprocess images
* calculate indices

We extract the following spectral indices for impervious surfaces, water bodies and vegetation:   
* **NDBI**
* **MNDWI**, **NDWI**,
* **NDVI**, **SAVI**


## Population density
* Use closest GHSL population dataset

## LST(Land surface temperature)
* Use Landsat Collection 2 Surface temperature bands
  * for Landsat 8 - band 10
  * for Landsat 4/5/7 - band 6

In [ ]:
# !pip install eemont
# !pip install ee
# !pip install geopandas
# !pip install rasterio
# !pip install earthengine-api
# !pip install earthengine-api --upgrade

In [ ]:
import ee
import eemont
import geopandas as gpd
import rasterio
import json
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-dianamarkovakn")

In [ ]:
aoi = ee.Geometry.Polygon([
  [[23.032164119466383, 42.91889685342199],
   [23.032164119466383, 42.39372184157957],

   [23.710569881185133, 42.39372184157957],
   [23.710569881185133, 42.91889685342199]]
]);

In [ ]:
spectral_incides = ['NDBI', 'NDBaI',	'MNDWI',	'NDWI',	'NDVI',	'SAVI']

In [ ]:
ST_bands = {'LE07': 'ST_B6',
            'LT05': 'ST_B6',
            'LC08': 'ST_B10', }

Red = {'LE07': 'SR_B3',
       'LT05': 'SR_B3',
       'LC08': 'SR_B4'}

N = {'LE07': 'SR_B4',
       'LT05': 'SR_B4',
       'LC08': 'SR_B5'}

Green = {'LE07': 'SR_B2',
         'LT05': 'SR_B2',
         'LC08': 'SR_B3'}

Blue = {'LE07': 'SR_B1',
        'LT05': 'SR_B1',
        'LC08': 'SR_B2'}

S1 = {'LE07': 'SR_B5',
         'LT05': 'SR_B5',
         'LC08': 'SR_B6'}

S2 = {'LE07': 'SR_B7',
         'LT05': 'SR_B7',
         'LC08': 'SR_B7'}


In [ ]:
id_list = [{'id': 'LC08_184030_20191015',
  'collection': 'LC08',
  'season': 'autumn',
  'tier': 'T1',
  'ghs': 'JRC/GHSL/P2023A/GHS_POP/2020'},]

In [ ]:
from collections import defaultdict
from datetime import datetime

with open('id_list.json', 'r') as f:
   id_list = json.load(f)


grouped = defaultdict(lambda: defaultdict(list))
for item in id_list:
    date_str = item['id'].split('_')[-1]
    date = datetime.strptime(date_str, "%Y%m%d")
    grouped[item['collection']][item['tier']].append(date)

# Display the results
for collection, tiers in grouped.items():
    print(f"Collection: {collection}")
    for tier, dates in tiers.items():
        print(f"  Tier: {tier}")
        for date in dates:
            print(f"    {date.strftime('%Y-%m-%d')}")



Collection: LT05
  Tier: T1
    1999-04-15
    2005-02-10
    2006-05-04
    2006-05-20
    2006-10-27
    2007-06-24
    2007-07-26
    2007-09-28
    2009-05-12
    2011-02-11
    2011-08-22
    2011-09-23
  Tier: T2
    2000-01-28
Collection: LE07
  Tier: T1
    1999-09-30
    2000-05-27
    2000-06-12
    2000-06-28
    2001-09-19
Collection: LC08
  Tier: T1
    2014-03-23
    2014-08-14
    2014-10-01
    2017-05-18
    2017-07-05
    2017-09-07
    2018-04-03
    2019-02-17
    2019-08-12
    2019-10-15


In [ ]:
indices = eemont.indices()
indices.SAVI.formula


'(1.0 + L) * (N - R) / (N + R + L)'

In [45]:
def collect_indices():
    for item in id_list:
        date_str = item['id'].split('_')[-1]
        date = datetime.strptime(date_str, "%Y%m%d")
        image_loc = f"LANDSAT/{item['collection']}/C02/{item['tier']}_L2/{item['id']}"
        print(f"image loc: {image_loc}, {date.strftime('%Y-%m-%d')}")



        scaled = ee.Image(image_loc).preprocess().reproject(crs='EPSG:32634', scale=120).resample('bilinear').clip(aoi)
        sat = ee.Image(image_loc).maskClouds().reproject(crs='EPSG:32634', scale=120).resample('bilinear').clip(aoi)
        pop = ee.Image(item['ghs']).reproject(crs='EPSG:32634', scale=120).resample('bilinear').clip(aoi).toFloat()
        image = sat

        G = sat.select(Green[item['collection']])
        B = sat.select(Blue[item['collection']])
        R = sat.select(Red[item['collection']])
        NIR = sat.select(N[item['collection']])
        SWIR1 = sat.select(S1[item['collection']])
        SWIR2 = sat.select(S2[item['collection']])
        T = sat.select(ST_bands[item['collection']])

        # Compute indices
        NDBI = (SWIR1.subtract(NIR).divide(NIR.add(SWIR1))).rename('NDBI')
        #NDBaI = (SWIR1.subtract(T).divide(SWIR1.add(T))).rename('NDBaI')
        MNDWI = (G.subtract(SWIR1).divide(G.add(SWIR1))).rename('MNDWI')
        NDWI = (G.subtract(NIR).divide(G.add(NIR))).rename('NDWI')
        NDVI = (NIR.subtract(R).divide(NIR.add(R))).rename('NDVI')
        SAVI = (NIR.subtract(R).divide(NIR.add(R).add(0.5)).multiply(1.5)).rename('SAVI')

        # Add indices to image
        image = image.addBands([NDBI, MNDWI, NDWI, NDVI, SAVI])

        # Optional: min/max check for indices
        for index in ['NDBI', 'MNDWI', 'NDWI', 'NDVI', 'SAVI']:
            min_max_dict = image.select(index).reduceRegion(
                reducer=ee.Reducer.minMax(),
                scale=120,
                geometry=aoi,
            ).getInfo()
            print(f"{index} min/max: {min_max_dict}")

        # Add population and LST to image
        image = image.addBands(pop.rename('population_count'))
        LST = scaled.select(ST_bands[item['collection']]).subtract(273.15).rename('LST').toFloat()
        image = image.addBands(LST)

        # Prepare the image for export
        name = item['id']
        image = image.select(['NDBI', 'MNDWI', 'NDWI', 'NDVI', 'SAVI', 'population_count', 'LST'])

        # Cast all bands to float
        image = image.cast({
            'NDBI': 'float', 'MNDWI': 'float',
            'NDWI': 'float', 'NDVI': 'float', 'SAVI': 'float',
            'population_count': 'float', 'LST': 'float'
        })

        # Export the image to Google Drive
        task = ee.batch.Export.image.toDrive(**{
            'image': image,
            'scale': 120,
            'description': name,
            'folder': 'data-indices-crs-fix',
            'region': aoi.bounds(),
        })
        task.start()


In [46]:
collect_indices()

image loc: LANDSAT/LT05/C02/T1_L2/LT05_184030_19990415, 1999-04-15
NDBI min/max: {'NDBI_max': 0.18288595859304038, 'NDBI_min': -0.751078132059198}
MNDWI min/max: {'MNDWI_max': 0.7599666993407006, 'MNDWI_min': -0.3182318260035544}
NDWI min/max: {'NDWI_max': 0.2535506269187731, 'NDWI_min': -0.4548091943762553}
NDVI min/max: {'NDVI_max': 0.4872950235277342, 'NDVI_min': -0.3253584646186826}
SAVI min/max: {'SAVI_max': 0.7309321127604055, 'SAVI_min': -0.4880352294616991}
image loc: LANDSAT/LE07/C02/T1_L2/LE07_184030_19990930, 1999-09-30
NDBI min/max: {'NDBI_max': 0.1795346990104908, 'NDBI_min': -0.267261115797084}
MNDWI min/max: {'MNDWI_max': 0.12682137075013492, 'MNDWI_min': -0.3393019420416681}
NDWI min/max: {'NDWI_max': 0.08146522443434186, 'NDWI_min': -0.435170153459736}
NDVI min/max: {'NDVI_max': 0.4552453027139875, 'NDVI_min': -0.06213710909646532}
SAVI min/max: {'SAVI_max': 0.6828568166620456, 'SAVI_min': -0.09320329012707224}
image loc: LANDSAT/LT05/C02/T2_L2/LT05_184030_20000128, 20

In [36]:
def print_crs_and_transform():
    for item in id_list:
        date_str = item['id'].split('_')[-1]
        date = datetime.strptime(date_str, "%Y%m%d")
        image_loc = f"LANDSAT/{item['collection']}/C02/{item['tier']}_L2/{item['id']}"
        print(f"Processing image: {image_loc}, Date: {date.strftime('%Y-%m-%d')}")

        # Load the image from Earth Engine
        landsat_image = ee.Image(image_loc)

        # Get projection info
        landsat_projection = landsat_image.projection()

        # Get CRS and Transform
        landsat_crs = landsat_projection.crs().getInfo()  # CRS as EPSG code
        landsat_transform = landsat_projection.transform().getInfo()  # Affine transform

        # Print CRS and Transform
        print(f"Image ID: {item['id']}")
        print(f"  - CRS: {landsat_crs != 'EPSG:32634'}")
        print(f"  - Transform: {landsat_transform}")
        print("\n")  # Add a newline for readability between images

In [39]:
import ee

# Initialize Earth Engine
ee.Initialize()

# Function to print CRS and transform
def print_pop_crs_and_transform(ghs_dataset):
    pop_image = ee.Image(ghs_dataset)
    pop_projection = pop_image.projection()

    # Get CRS and Transform
    pop_crs = pop_projection.crs().getInfo()  # CRS as EPSG code
    pop_transform = pop_projection.transform().getInfo()  # Affine transform

    # Print CRS and Transform
    print(f"GHS Dataset: {ghs_dataset}")
    print(f"  - CRS: {pop_crs}")
    print(f"  - Transform: {pop_transform}")
    print("\n")  # Add a newline for readability

# # Loop through id_list and print CRS and transform for each population dataset
# for item in id_list:
#     ghs_dataset = item['ghs']  # Assuming 'ghs' key holds the dataset path
#     print_pop_crs_and_transform(ghs_dataset)

In [38]:
# print_crs_and_transform()

In [41]:
# collect_indices()

In [47]:
tasks = ee.batch.Task.list()
for task in tasks[:30]:

    print(task.status())

{'state': 'READY', 'description': 'LC08_184030_20191015', 'priority': 100, 'creation_timestamp_ms': 1728908686492, 'update_timestamp_ms': 1728908691032, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DTOOIDA5JFH63O54SKOSLXJZ', 'name': 'projects/ee-dianamarkovakn/operations/DTOOIDA5JFH63O54SKOSLXJZ'}
{'state': 'READY', 'description': 'LC08_184030_20190812', 'priority': 100, 'creation_timestamp_ms': 1728908682533, 'update_timestamp_ms': 1728908684838, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HDJMZ7K4HSY2YLSXRAUNWLKV', 'name': 'projects/ee-dianamarkovakn/operations/HDJMZ7K4HSY2YLSXRAUNWLKV'}
{'state': 'READY', 'description': 'LC08_184030_20190217', 'priority': 100, 'creation_timestamp_ms': 1728908678997, 'update_timestamp_ms': 1728908683038, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'XLZFQSBJ6XYNWXTMWJNTIV25', 'name': 'projects/ee-dianamarkovakn/operations/XLZFQSBJ6XYNWXTMWJNTIV25'}
{'state': 'READY', 'description': 'LC08_184030_20180403

In [ ]:
def collect_pop():
    for year in range(1975, 2025, 5):
        pop = ee.Image(f"JRC/GHSL/P2023A/GHS_POP/{year}").clip(aoi)

        name = pop.id().getInfo()

        print(f"name: {name}")

        # Now we export and resample
        task = ee.batch.Export.image.toDrive(**{
            'image': pop,
            'description': name,
            'folder': 'population-ghs-aoi-clipped',
            'scale': 120,
            'region': aoi.bounds()
        })
        task.start()


In [ ]:
collect_pop()

name: 2020


In [ ]:
# tasks = ee.batch.Task.list()
# for task in tasks[:30]:
#     print(task.status())

In [ ]:
L5 = (ee.Image('LANDSAT/LT05/C02/T1/LT05_184030_19990415'))


In [ ]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()


aoi = ee.Geometry.Polygon([
[ [23.147785867237445, 42.591241793466885],
  [23.546040261768695, 42.591241793466885],
  [23.546040261768695, 42.83542772358794],
  [23.147785867237445, 42.83542772358794],
]]);

# Define urban and rural polygons
Urban = ee.Geometry.MultiPolygon(
        [[[[23.246904788408127, 42.688226341159925],
           [23.323809085283127, 42.65339079664303],
           [23.359514651689377, 42.69882458798372],
           [23.310076175126877, 42.71951119054903],
           [23.271624026689377, 42.71143919029933]]],
         [[[23.255144534501877, 42.74170377728034],
           [23.243471560869065, 42.710934655422136],
           [23.281923709306565, 42.71698880325915],
           [23.282610354814377, 42.72859093579189]]],
         [[[23.380114016923752, 42.70437532879963],
           [23.383547244462815, 42.674597366330666],
           [23.426805911455002, 42.6917592910562],
           [23.41787951985344, 42.7033661401078]]]])

Rural = ee.Geometry.MultiPolygon(
        [[[[23.46457141438469, 42.666771974480405],
           [23.469377932939377, 42.64657286301201],
           [23.49306720295891, 42.65869311722898],
           [23.481737552080002, 42.671568299263335]]],
         [[[23.254394538400174, 42.64365931309603],
           [23.258857734200955, 42.627241935261246],
           [23.282032020089627, 42.635830026978525],
           [23.267784125802518, 42.64530081266675]]]]);

landsat_image = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_184030_20191015').preprocess().clip(aoi)
st_band = landsat_image.select('ST_B10').subtract(273.15)


mean_urban = st_band.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Urban,
    scale=30,
    bestEffort=True
).get('ST_B10').getInfo()

mean_rural = st_band.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Rural,
    scale=30,
    bestEffort=True
).get('ST_B10').getInfo()


vis_params = {
    'min': mean_rural,
    'max': mean_urban,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red'],

}


Map = geemap.Map(center=[42.6977, 23.3219], zoom=10)

Map.addLayer(st_band, vis_params, "Landsat 8 ST Band (Celsius)")
Map.addLayer(aoi, {'color': 'black'}, "AOI")
Map.addLayer(Urban, {'color': 'green'}, "Urban Areas")
Map.addLayer(Rural, {'color': 'yellow'}, "Rural Areas")

print("Mean Surface Temperature in Urban Areas:", mean_urban)
print("Mean Surface Temperature in Rural Areas:", mean_rural)

Map



Mean Surface Temperature in Urban Areas: 23.984798299332656
Mean Surface Temperature in Rural Areas: 21.634435411484127


Map(center=[42.6977, 23.3219], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…